In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

Designed a comprehensive and interactive
modern dashboard for Superstore data
using Power BI.
Developed a sophisticated loan lending
model on the basis of credit score using
various machine learning algorithms and
deep learning architecture.
+91 9328162544
Ahmedabad, Gujarat
priyamvyas12345@gmail.com
Priyam Vyas
Programming Languages and Algorithms
Data Scientist | ML Engineer
CIMCOM Infotech PVT LTD.
14 May 2024 -  21 Jun 2024
Internship Experience
Developed and deployed a real-time data
processing pipeline for a smooth flow. 
SSC (2019-20): Divyapath Campus -
96.74 PR
Education
Skills & Competencies
Data and Quantitative Analysis               
Problem-Solving and Analytical Skills
Communication and Collaboration
Predictive Modelling 
Certifications
DSA using Java (Elite) - Nptel
NPTEL ( ELITE ) 
Getting Started with DL  
Nvidia Deep Learning Institute
Database Management Systems
NPTEL ( TOP 5% ) 
About
Always ready to learn and adapt to designing,
developing, and deploying scalable 

In [9]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Priyam Vyas"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Priyam Vyas. You are answering questions on Priyam Vyas's website, particularly questions related to Priyam Vyas's career, background, skills and experience. Your responsibility is to represent Priyam Vyas for interactions on the website as faithfully as possible. You are given a summary of Priyam Vyas's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Priyam Vyas. I’m a data scientist and ML engineer from Ahmedabad, India, currently pursuing a B.Tech in AI and ML at Charusat University. I’ve built projects like a loan lending model using deep learning and an interactive Power BI dashboard. I love working with data, Python, and AI—but outside of code, I’m just as passionate about learning new things and solving real-world problems.\n\n## LinkedIn Profile:\nDesigned a co

In [12]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [14]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [15]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [16]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [17]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [18]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [21]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a degree?"}]
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
reply = response.choices[0].message.content

In [22]:
reply

'I am currently pursuing a B.Tech degree in Artificial Intelligence and Machine Learning at Charusat University. My program is ongoing, and I have maintained a strong CGPA of 9.29 so far. If you have any questions about my coursework or projects during my studies, feel free to ask!'

In [23]:
evaluate(reply, "do you hold a degree?", messages[:1])

Evaluation(is_acceptable=True, feedback='The agent accurately answers the question by stating they are currently pursuing a B.Tech degree and provides additional relevant details like their university and CGPA. The tone is professional and engaging, inviting further questions, which aligns perfectly with the persona.')

In [24]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [26]:
def chat(message, history):
    if "degree" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [27]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The agent's response "Ar-charusat-university-day oday" is a garbled and incorrect version of "Charusat University." It fails to provide accurate information and is not professional or engaging. The correct response should clearly state "Charusat University."
